In [20]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.naive_bayes import *
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.discriminant_analysis import *
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [21]:
# read the data
red_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
white_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

In [22]:
# remove lines that have all values duplicated
red_wine.drop_duplicates(inplace=True, ignore_index=True)
white_wine.drop_duplicates(inplace=True, ignore_index=True)

In [23]:
# add quality_label column
for wine_type in [red_wine, white_wine]:
    # we are creating a new column called "quality_label", we define a range and associate that range with a label
    wine_type['quality_label'] = wine_type['quality'].apply(lambda value: 'low'
    if value <= 5 else 'medium'
    if value <= 7 else 'high')

    # here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
    wine_type['quality_label'] = pd.Categorical(wine_type['quality_label'],
    categories=['low', 'medium', 'high'])

In [24]:
# create a df with all wines

# add color of wine as parameter
red_wine['color'] = 'red'
white_wine['color'] = 'white'

# combine the wine dfs
wine = pd.concat([red_wine, white_wine], ignore_index=True)

# here we are transforming these labels into categrical data type (specific to pandas) instead of simple string
wine['color'] = pd.Categorical(wine['color'],
categories=['red', 'white'])

In [25]:
# use the rename method to change all columns names lowercase and add an underscore if they are made of 2 words
wine.rename(str.lower, axis='columns', inplace=True)  # make the names lowercase
wine.columns = wine.columns.str.replace(' ', '_')       # replace space with underscore in column names

In [26]:
wine

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality,quality_label,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,low,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,low,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,low,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,medium,red
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,low,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,medium,white
5316,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,low,white
5317,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,medium,white
5318,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,medium,white


In [27]:
# prepare wine data for model training

# drop color and quality column
white_wine_ml = white_wine.drop(['color', 'quality'], axis=1)

# replace the quality labels with numerical values
quality_codes = {'low' : 0, 'medium' : 1, 'high' : 2}
white_wine_ml['quality_label'].replace(quality_codes, inplace=True)

white_wine_ml

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_label
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,1
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.99490,3.18,0.47,9.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3956,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,1
3957,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0
3958,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,1
3959,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,1


In [28]:
# check at which point a machine learning model is actually better than just picking the most common value in the target variable
# if the model doesn't get 0.625347 accuracy, we are better off just saying everything is medium quality (1)

white_wine_ml['quality_label'].value_counts(normalize=True)

quality_label
1    0.625347
0    0.340318
2    0.034335
Name: proportion, dtype: float64

In [29]:
# split the data into features (X) and target variable (y)

def split_wine_data(wine_data_ml):
    X = wine_data_ml.drop(['quality_label'], axis=1) # features
    y = wine_data_ml['quality_label']  # target variable

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    return X_train, X_test, y_train, y_test

In [30]:
def evaluate_model(model_name, X_train, X_test, y_train, y_test):
# just runs a model and outputs it's evaluation metrics

    try: # if there is no errors
        model = model_name()
        model.fit(X_train, y_train.ravel())

        # make predictions on the test set
        y_pred = model.predict(X_test)

        # evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        classification_rep = classification_report(y_test, y_pred)
    except: # if there are errors, return 0 for evaluation
        accuracy = 0
        conf_matrix = 0
        classification_rep = 0
    
    return accuracy, conf_matrix, classification_rep

In [31]:
def evaluate_model_short(model_name, X_train, X_test, y_train, y_test):
# just runs a model and outputs it's evaluation metrics

    try: # if there is no errors
        model = model_name()
        model.fit(X_train, y_train.ravel())

        # make predictions on the test set
        y_pred = model.predict(X_test)

        # evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
    except: # if there are errors, return 0 for evaluation
        accuracy = 0
    
    return accuracy

In [32]:
# check metrics for multiple models
models = [LogisticRegression, LinearRegression, DecisionTreeClassifier, RandomForestClassifier, KNeighborsClassifier,
          SVC, GaussianNB, LinearDiscriminantAnalysis, AdaBoostClassifier]

def get_accuracy_for_models(models, wine_data_ml):

    X_train, X_test, y_train, y_test = split_wine_data(wine_data_ml)

    evaluation_results = []

    for model in models:
        accuracy = evaluate_model_short(model, X_train, X_test, y_train, y_test)

        # make the model name more readabkle in output
        evaluation_results.append([str(model).split('.')[-1].strip("'>"), accuracy])


    # convert evaluation metrics to df
    evaluation_results = pd.DataFrame(evaluation_results, columns=['model', 'accuracy'])

    # show the evaluation metrics ordered by accuracy
    return evaluation_results.sort_values('accuracy', ascending=False)


get_accuracy_for_models(models, white_wine_ml)

/Users/michalpasternak/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,model,accuracy
3,RandomForestClassifier,0.741488
7,LinearDiscriminantAnalysis,0.704918
0,LogisticRegression,0.664565
8,AdaBoostClassifier,0.663304
2,DecisionTreeClassifier,0.650694
6,GaussianNB,0.641866
4,KNeighborsClassifier,0.640605
5,SVC,0.619168
1,LinearRegression,0.000000


In [33]:
X_train, X_test, y_train, y_test = split_wine_data(white_wine_ml)

# initialize the model
forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
forest.fit(X_train, y_train.ravel())

# make predictions on the test set
y_pred = forest.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.7264
Confusion Matrix:
 [[188  90   0]
 [102 388   2]
 [  1  22   0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.68      0.66       278
           1       0.78      0.79      0.78       492
           2       0.00      0.00      0.00        23

    accuracy                           0.73       793
   macro avg       0.47      0.49      0.48       793
weighted avg       0.71      0.73      0.72       793



In [34]:
# prepare wine data for model training

# drop color and quality column
red_wine_ml = red_wine.drop(['color', 'quality'], axis=1)

# replace the quality labels with numerical values
quality_codes = {'low' : 0, 'medium' : 1, 'high' : 2}
red_wine_ml['quality_label'].replace(quality_codes, inplace=True)

red_wine_ml

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality_label
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,1
1355,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1356,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1357,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


In [35]:
red_wine_ml['quality_label'].value_counts(normalize=True)

quality_label
1    0.516556
0    0.470935
2    0.012509
Name: proportion, dtype: float64

In [36]:
get_accuracy_for_models(models, red_wine_ml)

/Users/michalpasternak/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,model,accuracy
3,RandomForestClassifier,0.753676
8,AdaBoostClassifier,0.709559
7,LinearDiscriminantAnalysis,0.702206
0,LogisticRegression,0.683824
2,DecisionTreeClassifier,0.658088
6,GaussianNB,0.650735
5,SVC,0.617647
4,KNeighborsClassifier,0.595588
1,LinearRegression,0.000000


In [37]:
X_train, X_test, y_train, y_test = split_wine_data(red_wine_ml)

# initialize the model
forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
forest.fit(X_train, y_train.ravel())

# make predictions on the test set
y_pred = forest.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

Accuracy: 0.7390
Confusion Matrix:
 [[ 95  29   0]
 [ 40 106   0]
 [  0   2   0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.77      0.73       124
           1       0.77      0.73      0.75       146
           2       0.00      0.00      0.00         2

    accuracy                           0.74       272
   macro avg       0.49      0.50      0.49       272
weighted avg       0.74      0.74      0.74       272



/Users/michalpasternak/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/michalpasternak/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/michalpasternak/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [38]:
# prepare wine data for model training

# drop color and quality column
wine_ml = wine.drop(['quality'], axis=1)

# replace the quality labels with numerical values
quality_codes = {'low' : 0, 'medium' : 1, 'high' : 2}
wine_ml['quality_label'].replace(quality_codes, inplace=True)

# INSERT ONE HOT ENCODING FOR COLORS HERE

wine_ml

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality_label,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1,red
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,0,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,1,white
5316,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0,white
5317,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,1,white
5318,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,1,white


In [39]:
wine_ml['quality_label'].value_counts(normalize=True)

quality_label
1    0.597556
0    0.373684
2    0.028759
Name: proportion, dtype: float64

In [40]:
get_accuracy_for_models(models, wine_ml)

,model,accuracy
0,LogisticRegression,0
1,LinearRegression,0
2,DecisionTreeClassifier,0
3,RandomForestClassifier,0
4,KNeighborsClassifier,0
5,SVC,0
6,GaussianNB,0
7,LinearDiscriminantAnalysis,0
8,AdaBoostClassifier,0


In [41]:
X_train, X_test, y_train, y_test = split_wine_data(wine_ml)

# initialize the model
forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
forest.fit(X_train, y_train.ravel())

# make predictions on the test set
y_pred = forest.predict(X_test)

# evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

# print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)

ValueError: could not convert string to float: 'white'

In [ ]:
df_list = []

for w in [wine_ml, red_wine_ml, white_wine_ml]:
    df_list.append(get_accuracy_for_models(models, w))

#concatenated_df = pd.concat(scores, axis=1, keys=['all', 'red', 'white'])
concatenated_df = pd.concat([df.set_index('model') for df in df_list], keys=['all', 'red', 'white'], axis=1)


concatenated_df